# Floods Cap Haitien Centrality Analysis
Betweeness centrality measures the flow along each edge of all origin-destination pairs in a graph.

In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
from shapely.geometry import Point

In [50]:
import os, sys

In [51]:
# Get reference to GOSTNets
import sys
sys.path.append(r'C:\repos\GOSTnets')
import GOSTnets as gn

In [52]:
# Get reference to GOSTNets
sys.path.append(r"C:\repos\peartree")
import peartree as pt

### Load graphs
Did not use the advanced snapped graphs because they included too many extra edges and increased the time too much for betweeness centrality to run

In [53]:
regular_graph = nx.read_gpickle(r"temp\clipped_cap_haitien_walk_w_ferries_via_osmnx.pickle")
flooded_graph = nx.read_gpickle(r"temp\flooded_clipped_cap_haitien_walk_w_ferries_via_osmnx.pickle")
regular_graph_w_gtfs = nx.read_gpickle(r"temp\4_G_walk_basic_cap_haitien_gtfs_merged_impute_walk_service0001.pickle")

### When the basic G_walk_flooded graph was merged with the GTFS feed there were two spots on the map where the GTFS feed is in a flooded area. The basic merged G_walk_flooded and GTFS output from Step 2 was loaded into QGIS and saved as a shapefile, then the edges that intersected a flood zone were manually deleted. Then the shapefile was saved as a CSV edges file. We will now read the CSV edges file and the merged G_walk_flooded and GTFS nodes CSV and build a new graph.

In [54]:
#need to load the manually modified gtfs graph
fpath = r"temp\modded_4_G_walk_flooded_basic_cap_haitien_gtfs_merged_impute_walk_service0001_edges.csv"

fpath_nodes = r"temp\4_G_walk_flooded_basic_cap_haitien_gtfs_merged_impute_walk_service0001_nodes.csv"

In [55]:
flooded_graph_w_gtfs = gn.edges_and_nodes_csv_to_graph(fpath_nodes, fpath, u_tag = 'stnode', v_tag = 'endnode', geometry_tag = 'WKT', largest_G = False)

In [56]:
flooded_graph_w_gtfs

In [57]:
gn.example_edge(flooded_graph_w_gtfs,5)

(330725194, 330743482, {'WKT': <shapely.geometry.multilinestring.MultiLineString object at 0x000001F867BCDD30>, 'field_1': 0, 'mode': 'walk', 'ref': nan, 'oneway': False, 'lanes': nan, 'trips': nan, 'name': 'Route usine', 'maxspeed': nan, 'orig_lengt': 527.9849999999999, 'highway': 'unclassified', 'bridge': nan, 'service': nan, 'length': 543.0702857142857, 'junction': nan, 'access': nan, 'est_width': nan, 'osmid': '30034450'})
(330725194, 2322274029, {'WKT': <shapely.geometry.multilinestring.MultiLineString object at 0x000001F867BCD310>, 'field_1': 1, 'mode': 'walk', 'ref': nan, 'oneway': False, 'lanes': nan, 'trips': nan, 'name': 'Route Boisdaut', 'maxspeed': nan, 'orig_lengt': 253.01622857142857, 'highway': 'unclassified', 'bridge': nan, 'service': nan, 'length': 260.24526367346937, 'junction': nan, 'access': nan, 'est_width': nan, 'osmid': '218402959'})
(330725194, 2322274029, {'WKT': <shapely.geometry.multilinestring.MultiLineString object at 0x000001F867BCD2E0>, 'field_1': 2, 'mod

In [58]:
#flooded_graph_w_gtfs.nodes()['POKXI_stop_0052']

In [59]:
#gn.example_node(flooded_graph_w_gtfs,15)

## Calculate betweeness centrality for all graphs

In [60]:
graphs = {'regular_graph': regular_graph,'flooded_graph': flooded_graph,'regular_graph_w_gtfs': regular_graph_w_gtfs,'flooded_graph_w_gtfs': flooded_graph_w_gtfs}

In [61]:
for key,value in graphs.items():
    print(type(value))

<class 'networkx.classes.multidigraph.MultiDiGraph'>
<class 'networkx.classes.multidigraph.MultiDiGraph'>
<class 'networkx.classes.multidigraph.MultiDiGraph'>
<class 'networkx.classes.multidigraph.MultiDiGraph'>


In [62]:
graphs_mod = graphs.copy()

In [63]:
for graph in graphs:
    print(graphs[graph])
    # This is an important step, edge betweeness will run on a MultiDiGraph, but the results will be wacky
    graphs_mod[graph] = ox.utils_graph.get_digraph(graphs[graph], weight="length")
    edges = nx.edge_betweenness_centrality(graphs_mod[graph], weight='length')
    for edge in edges:
        graphs_mod[graph][edge[0]][edge[1]]['edge_centrality'] = edges[edge]
    gn.save(graphs_mod[graph],f"cap_haitien_graph_criticality3_{graph}",r"temp", pickle=False, nodes=False)

## make a graph that shows the differences between the centrality values of the flooded graph and regular graph

In [64]:
flooded_graph_diff = graphs_mod['flooded_graph'].copy()

In [65]:
flooded_graph_diff

In [87]:
match_count = 0
no_match_count = 0

for edge in graphs_mod['flooded_graph'].edges:
    #print(edge)
    try:
        if graphs_mod['regular_graph'][edge[0]][edge[1]]:
            match_count += 1
            flooded_graph_diff[edge[0]][edge[1]]['diff_edge_centrality'] = graphs_mod['flooded_graph'][edge[0]][edge[1]]['edge_centrality'] - graphs_mod['regular_graph'][edge[0]][edge[1]]['edge_centrality']
    except:
        no_match_count += 1

(330725194, 330743482)
(330725194, 2322274029)
(330725194, 330742709)
(330725346, 614908804)
(330725346, 614908783)
(330725346, 2150615578)
(330725346, 616793092)
(330725641, 615022237)
(330725641, 632753456)
(330725641, 632753471)
(330726131, 619124324)
(330726131, 8195390533)
(330726131, 8146639476)
(330726263, 628932636)
(330726263, 2313137076)
(330726263, 2313053113)
(330726477, 2352163204)
(330726477, 330729331)
(330726477, 2259065931)
(330726602, 616897302)
(330726602, 331017365)
(330726602, 616897290)
(330726602, 331017381)
(330727247, 619334216)
(330727247, 7677260952)
(330727247, 619334219)
(330727395, 330749699)
(330727395, 632658946)
(330727395, 330728878)
(330727433, 614883034)
(330727433, 614882979)
(330727433, 614908972)
(330727498, 2313407624)
(330727498, 2330899823)
(330727498, 2333480368)
(330728440, 620878753)
(330728440, 330737299)
(330728440, 6725255758)
(330728440, 6273205324)
(330728809, 619785275)
(330728809, 2313307231)
(330728809, 2330851403)
(330728878, 330730

(616764996, 616764966)
(616764996, 617016650)
(616765002, 616764957)
(616765002, 616764996)
(616765002, 616764964)
(616765004, 616764957)
(616765004, 619785566)
(616765004, 619869096)
(616765006, 2301687396)
(616765006, 614472198)
(616765006, 617016650)
(616765006, 616765023)
(616765014, 616764962)
(616765014, 619869095)
(616765014, 619869080)
(616765021, 616764979)
(616765021, 616765031)
(616765021, 616764974)
(616765022, 616764979)
(616765022, 616765032)
(616765022, 619869097)
(616765023, 616765006)
(616765023, 331017387)
(616765023, 2301687399)
(616765024, 614472000)
(616765024, 616765026)
(616765024, 616765025)
(616765025, 616765024)
(616765025, 616765027)
(616765025, 614472002)
(616765026, 616765024)
(616765026, 617023148)
(616765026, 616765027)
(616765027, 331017389)
(616765027, 616765025)
(616765027, 616765026)
(616765030, 619869090)
(616765030, 622021181)
(616765030, 2277199495)
(616765030, 2272069105)
(616765031, 616764980)
(616765031, 616765033)
(616765031, 616765021)
(616765

(619785319, 619785318)
(619785319, 619785332)
(619785319, 619785316)
(619785320, 614608820)
(619785320, 614608817)
(619785320, 619785366)
(619785321, 614612527)
(619785321, 619785322)
(619785321, 619785315)
(619785321, 619785242)
(619785322, 619785321)
(619785322, 619811757)
(619785322, 5227084305)
(619785323, 619785349)
(619785323, 614607549)
(619785323, 5227084305)
(619785325, 619785334)
(619785325, 619785327)
(619785325, 614606961)
(619785327, 619785335)
(619785327, 619785328)
(619785327, 619785325)
(619785328, 619785336)
(619785328, 2296244288)
(619785328, 619785327)
(619785329, 619785348)
(619785329, 619785319)
(619785329, 619785315)
(619785329, 619785330)
(619785330, 619785332)
(619785330, 619785352)
(619785330, 619785329)
(619785330, 619785339)
(619785332, 2296244278)
(619785332, 619785330)
(619785332, 2296244285)
(619785332, 619785319)
(619785333, 619785305)
(619785333, 2296244278)
(619785333, 619785318)
(619785333, 619785336)
(619785334, 619785325)
(619785334, 619785307)
(6197

(2268694855, 2272038861)
(2268694855, 617022613)
(2268694855, 619785470)
(2269638379, 619812275)
(2269638379, 619812277)
(2269638379, 619812305)
(2269638379, 619812294)
(2269640953, 619394989)
(2269640953, 2269640954)
(2269640953, 2269768417)
(2269640954, 2269640953)
(2269640954, 2269640958)
(2269640954, 2269640956)
(2269640956, 619394989)
(2269640956, 2269640961)
(2269640956, 2269640954)
(2269640958, 2269640954)
(2269640958, 2269640961)
(2269640958, 6769991220)
(2269640961, 2269640956)
(2269640961, 619394982)
(2269640961, 2269640958)
(2269768417, 619394988)
(2269768417, 2269768422)
(2269768417, 2269640953)
(2269768422, 6769991239)
(2269768422, 619394989)
(2269768422, 2272879099)
(2269768422, 2269768417)
(2269777582, 2269777585)
(2269777585, 623954794)
(2269777585, 2269777582)
(2269777585, 2269780558)
(2269780558, 623966722)
(2269780558, 2269784685)
(2269780558, 2269777585)
(2269780558, 623966696)
(2269784683, 2269784685)
(2269784685, 2269784683)
(2269784685, 2269780558)
(2269784685, 2

(5227084957, 628930771)
(5227084957, 2296448760)
(5227085395, 2342238231)
(5227085395, 628930771)
(5227085395, 4622514825)
(5227085397, 4622514827)
(5227085672, 2342238223)
(5918811822, 619820607)
(5918811822, 616903136)
(5918811822, 616903208)
(5918811824, 616903136)
(5918811824, 2272149731)
(5918811824, 2324743382)
(6139396935, 620822340)
(6139396935, 620822314)
(6139396935, 6139396940)
(6139396940, 2210287462)
(6139396940, 614653288)
(6139396940, 6139396935)
(6139396953, 6139396954)
(6139396954, 6139396953)
(6139396954, 6139396983)
(6139396983, 6139396996)
(6139396983, 6139396954)
(6139396996, 6139396983)
(6139396996, 2275907617)
(6139396996, 620842033)
(6139397100, 6139397109)
(6139397106, 6139397109)
(6139397109, 6139397106)
(6139397109, 7340143609)
(6139397109, 6139397100)
(6264500322, 614909328)
(6264500322, 646749033)
(6264500322, 614617135)
(6273205324, 330736841)
(6273205324, 330728440)
(6273205324, 6273205339)
(6273205339, 6273205324)
(6279224820, 6279225023)
(6279224863, 62

In [67]:
gn.example_edge(flooded_graph_diff, 3)

(330725194, 330743482, {'osmid': 30034450, 'name': 'Route usine', 'highway': 'unclassified', 'oneway': False, 'length': 527.9849999999999, 'geometry': <shapely.geometry.linestring.LineString object at 0x000001F867BCDA60>, 'edge_centrality': 0.05082971349359684, 'diff_edge_centrality': 0.03519264785653121})
(330725194, 2322274029, {'osmid': 218402959, 'name': 'Route Boisdaut', 'highway': 'unclassified', 'oneway': False, 'length': 245.98799999999997, 'geometry': <shapely.geometry.linestring.LineString object at 0x000001F867BCDBE0>, 'edge_centrality': 0.0031544215161608655, 'diff_edge_centrality': -0.0016892986472215412})
(330725194, 330742709, {'osmid': [218402957, 30034598, 218402959], 'name': 'Route Boisdaut', 'highway': 'unclassified', 'oneway': False, 'length': 107.92500000000001, 'bridge': 'yes', 'geometry': <shapely.geometry.linestring.LineString object at 0x000001F86E9249A0>, 'edge_centrality': 0.0540955367799548, 'diff_edge_centrality': 0.03350180846608174})


In [98]:
gn.save(flooded_graph_diff,"cap_haitien_graph_criticality_flooded_graph_diff3",r"temp", pickle=False, nodes=True)

### Now calculate the differences between the flooded graph with GTFS vs the regular graph with GTFS

Found out that the nodes for the GTFS edges are different between the graphs because of PearTree. However it is possible to modify the relabel the nodes (along with associated edges) using the NetworkX function below:

In [69]:
graphs_mod

{'regular_graph': <networkx.classes.digraph.DiGraph at 0x1f866b7c700>,
 'flooded_graph': <networkx.classes.digraph.DiGraph at 0x1f8659a90d0>,
 'regular_graph_w_gtfs': <networkx.classes.digraph.DiGraph at 0x1f8659a9dc0>,
 'flooded_graph_w_gtfs': <networkx.classes.digraph.DiGraph at 0x1f863e62f70>}

In [70]:
regular_graph_w_gtfs_copy = graphs_mod['regular_graph_w_gtfs'].copy()

In [71]:
flooded_graph_w_gtfs_copy = graphs_mod['flooded_graph_w_gtfs'].copy()

In [72]:
regular_graph_w_gtfs_copy = nx.relabel_nodes(regular_graph_w_gtfs_copy, lambda x: (x[5:]) if (isinstance(x, str)) else x, copy=True)

In [73]:
flooded_graph_w_gtfs_copy = nx.relabel_nodes(flooded_graph_w_gtfs_copy, lambda x: (x[5:]) if (isinstance(x, str)) else x, copy=True)

In [74]:
gn.example_edge(flooded_graph_w_gtfs_copy)

(330725194, 330743482, {'WKT': <shapely.geometry.multilinestring.MultiLineString object at 0x000001F867BCDD30>, 'field_1': 0, 'mode': 'walk', 'ref': nan, 'oneway': False, 'lanes': nan, 'trips': nan, 'name': 'Route usine', 'maxspeed': nan, 'orig_lengt': 527.9849999999999, 'highway': 'unclassified', 'bridge': nan, 'service': nan, 'length': 543.0702857142857, 'junction': nan, 'access': nan, 'est_width': nan, 'osmid': '30034450', 'edge_centrality': 0.06618159897846307})


In [75]:
gn.example_edge(regular_graph_w_gtfs_copy)

(330725194, 330743482, {'osmid': 30034450, 'name': 'Route usine', 'highway': 'unclassified', 'oneway': False, 'length': 543.0702857142857, 'geometry': <shapely.geometry.linestring.LineString object at 0x000001F86E924970>, 'orig_length': 527.9849999999999, 'mode': 'walk', 'edge_centrality': 0.001418764938740537})


In [96]:
#flooded_graph_w_gtfs_copy[330725194][330743482]['edge_centrality'] - regular_graph_w_gtfs_copy[330725194][330743482]['edge_centrality'] 

In [97]:
#flooded_graph_w_gtfs_copy[330725194][330743482]

In [85]:
flooded_graph_w_gtfs_copy

In [92]:
match_count = 0
no_match_count = 0

for edge in flooded_graph_w_gtfs_copy.edges:
    #print(edge)
    try:
        if regular_graph_w_gtfs_copy[edge[0]][edge[1]]:
            match_count += 1
            flooded_graph_w_gtfs_copy[edge[0]][edge[1]]['diff_edge_centrality'] = flooded_graph_w_gtfs_copy[edge[0]][edge[1]]['edge_centrality'] - regular_graph_w_gtfs_copy[edge[0]][edge[1]]['edge_centrality'] 
    except:
        no_match_count += 1

In [93]:
match_count

11445

In [94]:
gn.example_edge(flooded_graph_w_gtfs_copy, 200)

(330725194, 330743482, {'WKT': <shapely.geometry.multilinestring.MultiLineString object at 0x000001F867BCDD30>, 'field_1': 0, 'mode': 'walk', 'ref': nan, 'oneway': False, 'lanes': nan, 'trips': nan, 'name': 'Route usine', 'maxspeed': nan, 'orig_lengt': 527.9849999999999, 'highway': 'unclassified', 'bridge': nan, 'service': nan, 'length': 543.0702857142857, 'junction': nan, 'access': nan, 'est_width': nan, 'osmid': '30034450', 'edge_centrality': 0.06618159897846307, 'diff_edge_centrality': 0.06476283403972254})
(330725194, 2322274029, {'WKT': <shapely.geometry.multilinestring.MultiLineString object at 0x000001F867BCD310>, 'field_1': 1, 'mode': 'walk', 'ref': nan, 'oneway': False, 'lanes': nan, 'trips': nan, 'name': 'Route Boisdaut', 'maxspeed': nan, 'orig_lengt': 253.01622857142857, 'highway': 'unclassified', 'bridge': nan, 'service': nan, 'length': 260.24526367346937, 'junction': nan, 'access': nan, 'est_width': nan, 'osmid': '218402959', 'edge_centrality': 0.020904813887358785, 'diff_

In [95]:
gn.save(flooded_graph_w_gtfs_copy,"cap_haitien_graph_criticality_flooded_graph_diff_w_gtfs3",r"temp", pickle=False, nodes=True)